# V23: TRICERATOPS(+ ) FPP with AO/contrast-curve constraint

Re-runs TRICERATOPS(+ ) using the same gated+detrended baseline as V22, but adds a **high-resolution imaging contrast curve** to constrain unresolved companions.

Input file (bundled with tutorial data):
- `docs/tutorials/data/tic188646744/PHARO_Kcont_plot.tbl`


In [ ]:
from __future__ import annotations

import contextlib
import json
import sys
from pathlib import Path

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

btv = sh.btv

step_id = '37_v23_triceratops_fpp_with_contrast_curve'
run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)

sectors = [82, 83]

# Detrend config chosen for stability (see V35)
bin_hours = 6.0
duration_buffer_factor = 2.0
sigma_clip = 5.0

# Step-specific cache directory: stores detrended LCs under flux_type='pdcsap'
cache_dir = run_out_dir / 'cache_pdcsap_detrended'

# Contrast curve (AO imaging)
contrast_curve_path = Path('docs/tutorials/data/tic188646744/PHARO_Kcont_plot.tbl')
contrast_curve = btv.load_contrast_curve_exofop_tbl(contrast_curve_path, filter='Kcont')

base = sh.load_dataset()
detrended_by_sector: dict[int, btv.LightCurve] = {}
for sector in sectors:
    detrended_by_sector[int(sector)] = sh.detrend_transit_masked_bin_median(
        base.lc_by_sector[int(sector)],
        bin_hours=bin_hours,
        duration_buffer_factor=duration_buffer_factor,
        sigma_clip=sigma_clip,
    )

from tess_vetter.api.datasets import LocalDataset

ds_det = LocalDataset(
    schema_version=base.schema_version,
    root=base.root,
    lc_by_sector=detrended_by_sector,
    tpf_by_sector={k: v for k, v in base.tpf_by_sector.items() if int(k) in sectors},
    artifacts=base.artifacts,
)

cache = btv.hydrate_cache_from_dataset(
    dataset=ds_det,
    tic_id=sh.TIC_ID,
    flux_type='pdcsap',
    cache_dir=cache_dir,
    sectors=sectors,
)

stitched_det = sh.stitch_pdcsap_sectors(ds_det, sectors)
depth_ppm, depth_err_ppm = sh.estimate_depth_ppm(stitched_det)

stdout_path = run_out_dir / 'triceratops_stdout.log'
stderr_path = run_out_dir / 'triceratops_stderr.log'

with (
    stdout_path.open('w', encoding='utf-8') as out,
    stderr_path.open('w', encoding='utf-8') as err,
    contextlib.redirect_stdout(out),
    contextlib.redirect_stderr(err),
):
    result = btv.calculate_fpp(
        cache=cache,
        tic_id=sh.TIC_ID,
        period=sh.PERIOD_DAYS,
        t0=sh.T0_BTJD,
        depth_ppm=depth_ppm,
        duration_hours=sh.DURATION_HOURS,
        sectors=sectors,
        stellar_radius=sh.STELLAR.radius,
        stellar_mass=sh.STELLAR.mass,
        preset='fast',
        replicates=3,
        seed=42,
        contrast_curve=contrast_curve,
    )

payload = {
    'cache_dir': str(cache_dir),
    'cached_flux_type': 'pdcsap',
    'sectors': sectors,
    'detrend': {
        'bin_hours': bin_hours,
        'duration_buffer_factor': duration_buffer_factor,
        'sigma_clip': sigma_clip,
    },
    'depth_ppm_used': float(depth_ppm),
    'depth_err_ppm_est': float(depth_err_ppm),
    'contrast_curve': {
        'path': str(contrast_curve_path),
        'filter': str(getattr(contrast_curve, 'filter', None)),
        'n_points': int(len(contrast_curve.separation_arcsec)),
        'sep_min_arcsec': float(contrast_curve.separation_arcsec.min()),
        'sep_max_arcsec': float(contrast_curve.separation_arcsec.max()),
        'dmag_min': float(contrast_curve.delta_mag.min()),
        'dmag_max': float(contrast_curve.delta_mag.max()),
    },
    'fpp_result': result,
    'logs': {
        'stdout': str(stdout_path),
        'stderr': str(stderr_path),
    },
}

json_name = 'triceratops_fpp_with_contrast_curve.json'
run_json_path = run_out_dir / json_name
run_json_path.write_text(json.dumps(payload, indent=2, sort_keys=True))
docs_json_path = None
if docs_out_dir is not None:
    docs_json_path = docs_out_dir / json_name
    docs_json_path.write_text(json.dumps(payload, indent=2, sort_keys=True))

summary = {
    'depth_ppm_used': round(float(depth_ppm), 3),
    'depth_err_ppm_est': round(float(depth_err_ppm), 3),
    'fpp': result.get('fpp'),
    'nfpp': result.get('nfpp'),
    'disposition': result.get('disposition'),
    'n_nearby_sources': result.get('n_nearby_sources'),
    'gaia_query_complete': result.get('gaia_query_complete'),
    'aperture_modeled': result.get('aperture_modeled'),
    'sectors_used': result.get('sectors_used'),
    'replicates': result.get('replicates'),
    'contrast_curve_filter': payload['contrast_curve']['filter'],
    'contrast_curve_n': payload['contrast_curve']['n_points'],
}

print(json.dumps(summary, indent=2, sort_keys=True))


<details>
<summary><b>Expected Output</b></summary>

Numbers may change slightly if upstream catalogs change. The key expectation is that the run succeeds and returns FPP/NFPP + scenario probabilities.

```text
{
  "aperture_modeled": true,
  "contrast_curve_filter": "Kcont",
  "contrast_curve_n": 97,
  "depth_err_ppm_est": 11.135,
  "depth_ppm_used": 210.635,
  "disposition": "VALIDATED",
  "fpp": 7.1e-05,
  "gaia_query_complete": true,
  "n_nearby_sources": 347,
  "nfpp": 0.0,
  "replicates": 3,
  "sectors_used": [
    82,
    83
  ]
}
```

</details>


In [ ]:
from pathlib import Path
import json

import matplotlib.pyplot as plt
import numpy as np

step_id = '37_v23_triceratops_fpp_with_contrast_curve'
fname = 'V23_triceratops_fpp_with_contrast_curve.png'

run_out_dir, docs_out_dir = sh.artifact_dirs(step_id=step_id)
run_path = run_out_dir / fname
docs_path = (docs_out_dir / fname) if docs_out_dir is not None else None

probs = {
    'planet': result.get('prob_planet'),
    'EB': result.get('prob_eb'),
    'BEB': result.get('prob_beb'),
    'NEB': result.get('prob_neb'),
    'NTP': result.get('prob_ntp'),
}
labels = list(probs.keys())
vals = [float(probs[k]) if probs[k] is not None else np.nan for k in labels]

fig, ax = plt.subplots(figsize=(7.5, 3.8), dpi=150)
ax.bar(labels, vals, color=['#2ca02c'] + ['#d62728'] * 4)
ax.set_ylim(0, 1)
ax.set_ylabel('Probability')
ax.set_title('V23: TRICERATOPS(+ ) scenario probabilities (with contrast curve)')
for i, v in enumerate(vals):
    if np.isfinite(v):
        ax.text(i, min(0.98, v + 0.03), f"{v:.3f}", ha='center', va='bottom', fontsize=9)

fpp = result.get('fpp')
nfpp = result.get('nfpp')
ax.text(
    0.98,
    0.98,
    f"FPP={fpp:.3g}\nNFPP={nfpp:.3g}" if fpp is not None and nfpp is not None else 'FPP/NFPP unavailable',
    ha='right',
    va='top',
    transform=ax.transAxes,
    bbox={'boxstyle': 'round,pad=0.25', 'fc': 'white', 'ec': '0.7'},
)
ax.text(
    0.02,
    0.98,
    'AO constraint: PHARO Kcont',
    ha='left',
    va='top',
    transform=ax.transAxes,
    bbox={'boxstyle': 'round,pad=0.25', 'fc': 'white', 'ec': '0.7'},
)
fig.tight_layout()
fig.savefig(run_path)
if docs_path is not None:
    fig.savefig(docs_path)
plt.close(fig)

print(
    json.dumps(
        {
            'run_plot_path': str(run_path),
            'docs_plot_path': str(docs_path) if docs_path is not None else None,
            'run_json_path': str(run_out_dir / 'triceratops_fpp_with_contrast_curve.json'),
            'docs_json_path': str(docs_out_dir / 'triceratops_fpp_with_contrast_curve.json') if docs_out_dir is not None else None,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_json_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/37_v23_triceratops_fpp_with_contrast_curve/triceratops_fpp_with_contrast_curve.json",
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/37_v23_triceratops_fpp_with_contrast_curve/V23_triceratops_fpp_with_contrast_curve.png",
  "run_json_path": "persistent_cache/tutorial_toi-5807-incremental/37_v23_triceratops_fpp_with_contrast_curve/triceratops_fpp_with_contrast_curve.json",
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/37_v23_triceratops_fpp_with_contrast_curve/V23_triceratops_fpp_with_contrast_curve.png"
}
```

</details>


## Plot

<img src="../artifacts/tutorial_toi-5807-incremental/37_v23_triceratops_fpp_with_contrast_curve/V23_triceratops_fpp_with_contrast_curve.png" width="820" />


<details>
<summary><b>Analysis</b></summary>

- **What this adds vs V22:** the AO contrast curve places an upper limit on the brightness of unresolved companions as a function of angular separation. This directly suppresses the probability of blended/background false positive scenarios that would require a bright nearby eclipsing system.
- **How to compare to V22:**
  - If `FPP` drops substantially when adding the contrast curve, that means V22's non-planet probability mass was partly supported by companions that are now ruled out by imaging.
  - If `FPP` barely changes, then (for this candidate and aperture model) imaging constraints are not the limiting factor; the result is driven by other priors/evidence.
- **Interpretation (this run):** adding the PHARO `Kcont` contrast curve reduces `FPP` from ~`2.62e-3` (V22) to ~`7.1e-5` while keeping `NFPP=0.0`, strengthening the planet-favored interpretation under the same preprocessing baseline.
- **What to take away:** the imaging constraint is informative here and materially reduces the allowed blend parameter space.
- **Cautions / caveats:**
  - This still assumes the depth is correct for the preprocessing choice; if depth is unstable, repeat the run for the plausible depth range (V24).
  - A contrast curve is filter-specific (here `Kcont`); it constrains companions via brightness contrasts but does not by itself prove the signal is on-target (localization still matters).
  - In crowded fields, ensure the sector/aperture assumptions used for the light curve are compatible with the imaging separation constraints.

</details>
